In [27]:
from model.mhformer import Model
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import ast
from torch.utils.data import Dataset, DataLoader
import torch.optim.lr_scheduler as lr_scheduler
from tqdm import tqdm
import sys
import matplotlib.pyplot as plt

In [28]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

BATCH_SIZE = 8
LEARNING_RATE = 1e-4

In [29]:
class LiftingDataset(Dataset):
    def __init__(self, dataset_path, device="cpu"):
        super().__init__()
        df = pd.read_csv(dataset_path)
        self.input, self.output = df['Input'], df['Output']
        self.input = torch.tensor([ast.literal_eval(x) for x in self.input], dtype=torch.float, device=device)
        self.output = torch.tensor([ast.literal_eval(x) for x in self.output], dtype=torch.float, device=device)

    def __len__(self):
        return self.input.shape[0]
    
    def __getitem__(self, index):
        return self.input[index], self.output[index]

In [30]:
train_dataset = LiftingDataset('train.csv', device)
test_dataset = LiftingDataset('test.csv', device)

train_dataloader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, BATCH_SIZE, shuffle=False)

In [31]:
class Args:
    def __init__(self):
        self.frames = 30
        self.n_joints = 133
        self.channel = 256
        self.out_joints = 133
        self.layers = 4
        self.d_hid = 512

args = Args()
model = Model(args)

In [32]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer)

In [33]:
epochs = 100
train_loss_values = []
test_loss_values = []
last_train, last_test, best_train, best_test = None, None, sys.maxsize, sys.maxsize

for epoch in tqdm(range(epochs)):
    avg_train_loss, avg_test_loss = 0, 0

    model.train()

    for X, y in train_dataloader:
        optimizer.zero_grad()

        pred = model(X)

        loss = criterion(pred, y)

        avg_train_loss += loss.item()

        loss.backward()
        optimizer.step()

    model.eval()

    with torch.inference_mode():
        for X, y in test_dataloader:
            pred = model(X)

            loss = criterion(pred, y)

            avg_test_loss += loss.item()
    
    avg_train_loss /= len(train_dataloader)
    avg_test_loss /= len(test_dataloader)

    last_train = avg_train_loss
    last_test = avg_test_loss

    if avg_test_loss < best_test:
        best_train = avg_train_loss
        best_test = avg_test_loss

    print(f"Epoch: {epoch} | Train Loss: {avg_train_loss} | Test Loss: {avg_test_loss}")
    scheduler.step(avg_train_loss)

    train_loss_values.append(avg_train_loss)
    test_loss_values.append(avg_test_loss)

  0%|          | 0/100 [00:00<?, ?it/s]


ValueError: not enough values to unpack (expected 4, got 3)

In [ ]:
plt.figure(figsize=(12, 6))
plt.title("Loss")
plt.plot(range(epochs), train_loss_values, label="Train")
plt.plot(range(epochs), test_loss_values, label="Test")
plt.legend()